In [25]:
import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import google_analytics_reporting_api as ga
import os
import re
import string
import glob

Start_Date_overall='2018-01-01'
End_Data_overall='2018-07-22'

week="_Year_2018"

today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Google AdWords

In [26]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [27]:
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","AdvertisingChannelType","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)

In [28]:
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","AdvertisingChannelType","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)

In [29]:
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","AdvertisingChannelType","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)

In [30]:
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]
# Filter out campaign contains "Loom" and "Leaf" and "Zenhaven" from Saatva account
Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']
# Keep only campaign labeled as "[SM]" in Saatva Competitor account
Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type

In [31]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

In [32]:
Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
len(Saatva_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0

In [33]:
# All campaing is able to differentiate from the name about brand and non-brand, so no need to go to keywords level
'''
Saatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)
Saatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva branding_keywords.csv",dtype=str)
Saatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)

Saatva_keywords=Saatva_keywords[~((Saatva_keywords['Campaign'].str.contains("Loom")) | (Saatva_keywords['Campaign'].str.contains("Leaf")) | (Saatva_keywords['Campaign'].str.contains("Zenhaven")))]
Saatva_keywords['Campaign_Type']=np.where(Saatva_keywords['Search keyword'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].astype(int)
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].astype(int)
Saatva_keywords=Saatva_keywords.groupby(['Campaign','Campaign_Type'])['Impressions','Clicks'].sum()
Saatva_keywords.reset_index(inplace=True)
Saatva_keywords=Saatva_keywords.sort_values(['Campaign','Impressions'])
Saatva_keywords=Saatva_keywords.drop_duplicates('Campaign')
Saatva_Campaign_Type=Saatva_keywords[['Campaign','Campaign_Type']]

del Saatva_Click_Search_2['Campaign_Type']
Saatva_Click_Search_2=pd.merge(Saatva_Click_Search_2,Saatva_Campaign_Type,on="Campaign",how="left")

Saatva_Search_Click=Saatva_Click_Search_1.append(Saatva_Click_Search_2)
Saatva_Search_Click['Campaign_Type']=Saatva_Search_Click['Campaign_Type'].fillna("All_0_Values_and_No_Kws")
'''

'\nSaatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)\nSaatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva branding_keywords.csv",dtype=str)\nSaatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)\n\nSaatva_keywords=Saatva_keywords[~((Saatva_keywords[\'Campaign\'].str.contains("Loom")) | (Saatva_keywords[\'Campaign\'].str.contains("Leaf")) | (Saatva_keywords[\'Campaign\'].str.contains("Zenhaven")))]\nSaatva_keywords[\'Campaign_Type\']=np.where(Saatva_keywords[\'Search keyword\'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].apply(lambda x: int(x.replace(",","")))\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].astype(int)\nSaatva_keywords[\'Clicks\']=Saatva_keywords[\'Clicks\'].apply(lambda x: int(x.replace(",","")))

In [34]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)

In [35]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.xlsx",engine='xlsxwriter')

In [36]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [37]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day"])['Impressions','Clicks', 'Cost'].sum()
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]

Saatva_Agg_No_NA.to_excel(writer,"clean data by day",index=False)

In [38]:
writer.save()

In [39]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date"],ascending=[True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.csv",index=False)

# Sales

In [40]:
klipfolio_data=pd.read_excel("/home/jian/Projects/Saatva/Call_Analysis/data/Transaction_with_phone_2018_to_May.xlsx",sheetname="Sales_By_Brand",dtype=str)
klipfolio_data=klipfolio_data.rename(columns={"Zip_Code":"zip_cd"})
klipfolio_data['Date']=klipfolio_data['Date'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0],"%Y-%m-%d").date())
klipfolio_data['Sales']=klipfolio_data['Sales'].astype(float)
klipfolio_data['Phone_1']=klipfolio_data['Phone_1'].apply(lambda x: x.replace("-",""))
klipfolio_data['Phone_2']=klipfolio_data['Phone_2'].apply(lambda x: x.replace("-",""))
klipfolio_data['Phone_1']=klipfolio_data['Phone_1'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
klipfolio_data['Phone_2']=klipfolio_data['Phone_2'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
klipfolio_data['Phone_1']=klipfolio_data['Phone_1'].apply(lambda x: x.replace(" ",""))
klipfolio_data['Phone_2']=klipfolio_data['Phone_2'].apply(lambda x: x.replace(" ",""))

del klipfolio_data['CSR']
del klipfolio_data['PressOne_Rep_Name']

In [41]:
klipfolio_data_2_folder="/home/jian/Projects/Saatva/Transaction_Data/2018_weekly_klipfolio_with_Phone/"
klipfolio_data_2_file_list=glob.glob(klipfolio_data_2_folder+"*.csv")
for file in klipfolio_data_2_file_list:

    df=pd.read_csv(file,dtype=str)
    df=df[['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type',
                                   'Phone 1','Phone 2','PressOne Rep Name','Brand']]

    df=df.rename(columns={"Order ID":"Order_id","Shipping Zip":"zip_cd","Product Revenue":"Sales",
                                                     "Order Type":"Source","Phone 1":"Phone_1","Phone 2":"Phone_2",
                                                     "PressOne Rep Name":"PressOne_Rep_Name"})
    
    df=df.fillna("nan")
    df=df[df['Order_id']!="nan"]
    df['Date']=df['Date'].apply(lambda x: x.replace("-April-","-Apr-").replace("-March-","-Mar-").replace("-June-","Jun"))
    df['Date']=np.where(len(df['Date'])<df['Date'].apply(lambda x: len(x)).max(),"0"+df['Date'],df['Date'])
    df['Date']=df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
    df['Sales']=df['Sales'].astype(float)
    df['Phone_1']=df['Phone_1'].apply(lambda x: x.replace("-",""))
    df['Phone_2']=df['Phone_2'].apply(lambda x: x.replace("-",""))
    df['Phone_1']=df['Phone_1'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
    df['Phone_2']=df['Phone_2'].apply(lambda x: re.sub('['+string.punctuation+']', '', x))
    df['Phone_1']=df['Phone_1'].apply(lambda x: x.replace(" ",""))
    df['Phone_2']=df['Phone_2'].apply(lambda x: x.replace(" ",""))
    

    klipfolio_data=klipfolio_data[klipfolio_data['Date']<=(df['Date'].min()+datetime.timedelta(days=1))]
    klipfolio_data=klipfolio_data.append(df)
    klipfolio_data=klipfolio_data.drop_duplicates()
    
transaction_data=klipfolio_data[klipfolio_data['Brand']=="Saatva"].reset_index()
del transaction_data['index']
del transaction_data['PressOne_Rep_Name']
transaction_data['Time']=transaction_data['Time'].apply(lambda x: x.zfill(8))
transaction_data=transaction_data.drop_duplicates()
transaction_data=transaction_data.reset_index()
del transaction_data['index']

Transaction_data_sales=transaction_data[['Order_id','Date','Sales','zip_cd']].drop_duplicates()
Transaction_data_sales=Transaction_data_sales.drop_duplicates(['Order_id'])


In [42]:
US_Zips=pd.read_csv("/home/jian/Docs/EASI/2018-06-25/ZIP4_18_DATA_A2_CSV/Total_HH_POP_by_Zip_JL_20180723.csv",dtype=str)
Transaction_data_sales=Transaction_data_sales[Transaction_data_sales['zip_cd'].isin(US_Zips['ZIP_CODE'])]
Transaction_data_sales_by_Day=Transaction_data_sales.groupby(['Date'])['Sales'].sum().to_frame().reset_index()

In [43]:
Ad_Data_by_Day=AdWords_output.groupby(['Date'])['IMP','Clicks','Cost'].sum().reset_index()

In [44]:
Ad_Data_by_Day['Date']=Ad_Data_by_Day['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())

In [45]:
output_by_day=pd.merge(Ad_Data_by_Day,Transaction_data_sales_by_Day,on="Date",how="left")

In [46]:
output_by_day['week_num']=output_by_day['Date'].apply(lambda x: "week_"+str(int(np.floor((x-datetime.date(2018,1,1)).days/7)+1)))
output_by_day['ROI']=output_by_day['Sales']/output_by_day['Cost']
output_by_day['ROI']=output_by_day['ROI'].apply(lambda x: round(x,2))

In [47]:
output_by_week=output_by_day.groupby(['week_num'])['IMP','Clicks','Cost','Sales'].sum().reset_index()
output_by_week['ROI']=output_by_week['Sales']/output_by_week['Cost']
output_by_week['ROI']=output_by_week['ROI'].apply(lambda x: round(x,2))
output_by_week['order']=output_by_week['week_num'].apply(lambda x: int(x.split("_")[1]))
output_by_week=output_by_week.sort_values(['order'])
del output_by_week['order']

In [48]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.xlsx",engine='xlsxwriter')
output_by_week.to_excel(writer,"ROI_Data_weekly",index=False)
output_by_day.to_excel(writer,"ROI_Data_daily",index=False)
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)
writer.save()